In [10]:
import os

In [11]:
os.chdir("../")

In [12]:
pwd

'c:\\Users\\shrin\\Documents\\Python_Scripts\\adept\\Job-Classification'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig():
    root_dir: Path
    train_source_URL: str
    test_source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [26]:
from jobClassification.constants import *
from jobClassification.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            train_source_URL=config.train_source_URL,
            test_source_URL=config.test_source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [28]:
import os
import urllib.request as request  
import zipfile
from jobClassification.logging import logger
from jobClassification.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self, url_path):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = url_path,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        print(unzip_path)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        os.remove(self.config.local_data_file)

In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file(url_path=data_ingestion_config.train_source_URL)
    data_ingestion.extract_zip_file()
    data_ingestion.download_file(url_path=data_ingestion_config.test_source_URL)
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-27 01:47:08,842: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-27 01:47:08,845: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-27 01:47:08,847: INFO: common: created directory at: artifacts]
[2023-11-27 01:47:08,848: INFO: common: created directory at: artifacts/data_ingestion]


[2023-11-27 01:47:09,905: INFO: 215520230: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 22626674
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "1acc769fdc9eb7153bf67a9889a8f7ef45b464256c8f1fb4004f60441698d898"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1F64:35BD:169A8A1:1BD4C75:65643383
Accept-Ranges: bytes
Date: Mon, 27 Nov 2023 06:47:09 GMT
Via: 1.1 varnish
X-Served-By: cache-atl18453-ATL
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1701067630.910866,VS0,VE1
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 94a725cfaa308e96f8697667dab00c1533e35174
Expires: Mon, 27 Nov 2023 06:52:09 GMT
Source-Age: 137

]
artifacts/data_ingestion
[2023-11-27 01:47:1